In [ ]:
from datetime import datetime, timedelta
from google.colab import drive
import geopandas as gpd
import pandas as pd
import calendar
import geemap
import ee

In [ ]:
# Authenticated Earth Engine
ee.Authenticate()
ee.Initialize(project='try-spasial')

# Move assets to ImageCollection

In [ ]:
# Folder sumber (tempat asset berada)
source_folder = 'projects/try-spasial/assets/sentinel1'

# Folder tujuan (tempat asset akan dipindahkan)
destination_folder = 'projects/try-spasial/assets/collection'

print(f"Source folder: {source_folder}")
print(f"Destination folder: {destination_folder}")

def list_and_move_assets(source_folder, destination_folder):
    """List all assets in source folder and move them to ImageCollection with specified scale and dimensions"""
    try:
        # List all assets in the source folder
        asset_list = ee.data.listAssets({'parent': source_folder})

        if not asset_list.get('assets'):
            print("❌ No assets found in source folder!")
            return []

        print(f"Found {len(asset_list['assets'])} assets in the source folder:")
        print("=" * 50)

        # Membuat ImageCollection dari semua asset di folder
        images = []
        for asset in asset_list['assets']:
            asset_id = asset['id']
            image = ee.Image(asset_id)
            images.append(image)
            print(f"Asset: {asset_id}")

        # Gabungkan semua citra menjadi ImageCollection
        image_collection = ee.ImageCollection(images)

        # Menyimpan citra
        for image in images:
            asset_name = image.getInfo()['id'].split('/')[-1]  # Mengambil nama asset
            destination_path = f"{destination_folder}/{asset_name}"
            print(f"Moving asset to: {destination_path}")

            scale = 10  # Resolusi skala dalam meter
            region = image.geometry().bounds()  # Ambil batas geometri citra untuk menentukan region

            task = ee.batch.Export.image.toAsset(
                image=image,
                description=f"Export_{asset_name}",
                assetId=destination_path,
                scale=scale,
                region=region,
                maxPixels=int(1e13)
            )
            task.start()
            print(f"Export task started for {asset_name}")

        return image_collection

    except Exception as e:
        print(f"❌ Error processing assets: {str(e)}")
        return []

# Run the function to move files
list_and_move_assets(source_folder, destination_folder)